In [3]:
from Numberjack import *
from math import *
from nose.tools import assert_equal, ok_, assert_almost_equal
from utils import *
from tests import *
from copy import *

<div class="alert alert-warning">
**Important:** You will need SCIP to be working with Numberjack for this assignment. If you are having difficulties, refer to the Numberjack page on the course wiki.
   

If Numberjack isn't importing, restart the server by going to File->Hub Control Panel->Stop Server->Start Server.
    
Also make sure not to delete any of the answer cells or you will need to fetch a fresh copy of the assignment.
</div>


# Problem Set 07: Model Based Diagnosis and Conflicted Directed Search

In this problem set, we will model the probabilistic diagnosis problem as an OptSat problem that can be solved using Numberjack. Then we will practice transforming conflicts into diagnoses and obtaining the kernel diagnosis.

We will **not** be implementing a conflict-directed search algorithm directly. If that's something you want to learn, check out 16.412! However, it is important to understand the solvers we are calling using Numberjack have these conflict-directed search techniques built into them.

0. [Credit for Contributors (required)](#contributors)

1. [Model Based Diagnosis (40 points)](#problem1)

2. [Generating Kernel Diagnoses from Conflicts (60 points)](#problem2)
 
    
**100 points** total for Problem Set 07


## <a name="contributors"></a> Credit for Contributors

List the various students, lecture notes, or online resouces that helped you complete this problem set:

Ex: I worked with Bob on the cat activity planning problem.

<div class="alert alert-info">
Write your answer in the cell below this one.
</div>

Checked part 1 method with Lorenzo and Reinaldo, and got help on all parts and verified answers with Shashank.


## Part 1: Model Based Diagnosis  <a name="problem1"></a>

For the purposes of this problem, we have specialized Numberjack's variable classes into two new subclasses; `AndVariable` and `XOrVariable`, used to indicate the status of AND gates and XOR gates, respetively. For the purposes of this exercise, components either behave correctly or are in an 'unknown' failure state, where any output can occur. The variables take the value of `1` when the component is behaving correctly, and `0` when they are in the unknown failure state.

`AndVariable` and `XOrVariable` behave and can be used exactly like standard Numberjack binary variables, but they can be distinguished by their type, which we use in the objective function.

### Problem 1 
Consider AND and XOR gates as shown below. We assign variables to the component state (A1 or X1), as well as the inputs (in1 and in2) and outputs (out). Write a sentence in the language of propositional logic that encodes the behavior of the gates. Assume, as above, that a value of `1` assigned to the component state means it is working correctly, while `0` implies it is in an unknown failure state.

<img src='ANDgate.png'/> <img src='XORgate.png'/> 

<div class="alert alert-info">
Write your propositional sentences for the two gates below.
</div>

**Propositional sentence for AND gate: (10 points)**

state implies (out iff (in1 and in2))

**Propositional sentence for XOR gate: (10 points)**

state implies (out iff ((in1 and not(in2)) or (not(in1) and in2))

### Problem 2 (20 points)

Now implement the functions `add_AndVariable` and `add_OrVariable` below, which add the behavior you just described to a Numberjack model. These functions take the following inputs:

* `model`: A Numberjack defined with the `Model()` function.
* `state`: An `AndVariable` or `XOrVariable` that describes the state of the component.
* `in1`, `in2`: Binary variables describing the inputs to the component.
* `out`: Binary variable describing the output of the component.

They both output the model that was input with the behavior of their corresponding component added.

You can add a constraint to a Numberjack model using commands such as

```python
model += (V1 | (V2 == 0)) & V3
```

Which we can interpet as the CNF statement

$$(V_{1} \, \vee \, \bar{V_{2}}) \, \wedge \, V_{3}$$

<div class="alert alert-info">
Implement `add_AndVariable` and `add_XORVariable` below.
</div>

In [34]:
def add_AndVariable(model,state,in1,in2,out):
    model += (state==0) | (((out==0) | (in1 & in2)) & ((in1==0) | (in2==0) | out))
    return model
    

In [35]:
test_add_AndVariable(add_AndVariable)

Test passed!!

In [36]:
def add_XOrVariable(model,state,in1,in2,out):
    model += (state==0) | ((((out==0) | ((in1 & (in2==0)) | ((in1==0) & in2))) & (((in1==0) | in2) & (in1 | (in2==0))) | out))
    return model
    

In [37]:
test_add_XOrVariable(add_XOrVariable)

Test passed!!

Now we're ready to model the Boolean polycell example from class. In this model, XOR gates are 3% likely to fail, and AND gates are 2% likely to fail. All components are assumed to fail independently. Feel free to play with the failure probabilities to see how the solution changes. (Note that because of the formulation, probabilities of zero will fail.)

<img src='Polycell.PNG'/>

In [38]:
A = Variable('A')
B = Variable('B')
C = Variable('C')
D = Variable('D')
E = Variable('E')
X = Variable('X')
Y = Variable('Y')
Z = Variable('Z')
F = Variable('F')
G = Variable('G')

# Components
A1 = AndVariable('A1')
A2 = AndVariable('A2')
A3 = AndVariable('A3')
X1 = XOrVariable('X1')
X2 = XOrVariable('X2')

model = Model()

# Component constraints
model = add_AndVariable(model,A1,A,C,X)
model = add_AndVariable(model,A2,B,D,Y)
model = add_AndVariable(model,A3,C,E,Z)
model = add_XOrVariable(model,X1,X,Y,F)
model = add_XOrVariable(model,X2,Y,Z,G)
# Maximize probability objective
(model,obj) = add_diagnosis_objective(model,AND_fail=0.02,XOR_fail=0.03)

# Problem setup
model += (A == 1, B == 1, C == 1, D == 0, E == 1, F == 0, G == 1)


solver = model.load('SCIP')
solver.solve()

if solver.is_sat():
    # A solution has been found
    print("Solution found!!")
    for v in model.variables:
        if type(v).__name__ in ['AndVariable','XOrVariable']:
            print("{} = {} ".format(v.name(), v.get_value()))
    print("Probability of solution = {0:.3f}".format(exp(obj.get_value())))
else:
    # No solution exists
    print("No solution found.")

Solution found!!
A1 = 1 
A2 = 1 
A3 = 1 
X1 = 0 
X2 = 1 
Probability of solution = 0.027


## Part 2: Generating Kernel Diagnoses from Conflicts  <a name="problem2"></a>

Typically, conflicts are extracted over the course of search. For now, let's assume we've been given a conflict, which will take the form of a set of two-element tuples. The first element of each tuple is the name of the decision variable for a component, and the second will be an assignment. As in part 1, we will use `1` to indicate a functional state, and `0` to indicate an unknown state. 

For example, we write the conflict {A1=G,A2=G,X1=G} as:

```python
set([('A1',1),('A2',1),('X1',1)])
```

### Problem 3 (10 points)

How, conceptually, might we verify that a given set of tuples is a conflict?

<div class="alert alert-info">
Answer the question in the space below.
</div>

To verify, conceptually, that a given set of tuples is a conflict, we first need to define terms. First, a conflict is an assignment of the variables, like set([('A1',1),('A2',1),('X1',1)]). We also have a set of observations consisting of the inputs and outputs to the variables as we see on the polycell. Finally, there is a model of the system that describes how the system should behave. When the tuple (assignment of variables) is fed to the model with the set of observations, it is a conflict when the model cannot find a solution (meaning it cannot find how the assignment can work given the observations). To verify this conceptually, we would need to look at the model (like the polycell model above) and assign the variables with respect to the given set of tuples. Then, we would need to systematically assign either 1 or 0 to the inputs and outputs with respect to these assignments. If the input and outputs created by the assignments (given in the tuple set) do not match the observations, then we have a conflict.

In a similar way, we will store each kernel diagnosis as a set, meaning that any assignment that is a superset of the kernel diagnosis satisfies all conflicts found so far. For example, a kernel diagnosis

```python
set([('A2',0),('X2',0)])
```

encodes that {A2=U,X2=U} solves all conflicts.

We will store all the kernel diagnoses as a list of sets, where each set is an individual kernel diagnosis.

### Problem 4 (40 points)
Implement the function `update_kernel_diagnoses` which updates the kernel diagnoses when a new conflict has been discovered. It takes as input:

* `kernel_diagnoses`: A list of sets representing the currently held kernel diagnoses.
* `conflict`: A set representing a conflict.

It outputs an updated set of kernel diagnoses that satisfy all conflicts. Remember you must handle the case where `kernel_diagnoses` is an empty list (no conflicts have yet been discovered).

You will receive no credit if you use existing set covering functions.

**Hint:** When we use sets the element ordering doesn't matter, so we can test equality between sets with `s1 == s2`. Add elements with `s1.add(el)`, and remove elements with `s1.remove(el)`. Test whether `s1` is a subset of `s2` with `s1.issubset(s2)`.

<div class="alert alert-info">
Implement `update_kernel_diagnoses`.
</div>

In [19]:
def update_kernel_diagnoses(kernel_diagnoses,conflict):
    Next_Kernels = [] #output
    
    candidates = set()
    for c in conflict: 
        if c[1] == 1:
            temp = (c[0],0) # flips from G to U
        else:
            temp = (c[0],1) # flips from U to G
            
        candidates.add(temp) #adds diagnoses of conflict
    
    if not kernel_diagnoses:
        for i in candidates:
            Next_Kernels.append(set([i])) # takes candidates and puts into empty set, the first kernel
        return Next_Kernels
        
    Combined_Kernels = []      
    for k in kernel_diagnoses:
        for can in candidates:
            combined_set = set(k)
            combined_set.add(can)
            Combined_Kernels.append(combined_set)
            
    remove_list = []
    for ind in Combined_Kernels:
        if len(ind) == 1:
            Next_Kernels.append(ind)
            remove_list.append(ind)
    
    for bye in remove_list:
        Combined_Kernels.remove(bye)
    #pdb.set_trace() 
    pair_remove_list = []
    for pair in Combined_Kernels:
        for blah in remove_list:
            if blah.issubset(pair):
                pair_remove_list.append(pair)
    #pdb.set_trace()        
    for bye_bye in pair_remove_list:
        if bye_bye in Combined_Kernels:
            Combined_Kernels.remove(bye_bye)
        else:
            continue #skip duplicate values
        
    for kernel in Combined_Kernels:
        Next_Kernels.append(kernel)      
    
    return Next_Kernels
    

In [20]:
test_update_kernel_diagnoses(update_kernel_diagnoses)

Test passed!!

In [23]:
kernel_diagnoses = [{('X1', 0)}, {('A2', 0)}, {('A1', 0)}]
conflict = {('A1', 1), ('X2', 1), ('X1', 1), ('A3', 1)}
kernel_diagnoses = update_kernel_diagnoses(kernel_diagnoses,conflict)
print(kernel_diagnoses)
# ok_(len(kernel_diagnoses)==3,msg="Wrong length of kernel_diagnoses.")
# ok_(set([('A1',1)]) in kernel_diagnoses,msg="Missing element of kernel_diagnoses.")
# ok_(set([('A2',0)]) in kernel_diagnoses,msg="Missing element of kernel_diagnoses.")
# ok_(set([('X1',0)]) in kernel_diagnoses,msg="Missing element of kernel_diagnoses.")

[{('X1', 0)}, {('A1', 0)}, {('X2', 0), ('A2', 0)}, {('A3', 0), ('A2', 0)}]


### Problem 5 (10 points)
Finally, use `update_kernel_diagnoses` to implement `all_kernel_diagnoses`. This will take as input a list of conflicts, and return a list of all kernel diagnoses.

<div class="alert alert-info">
Implement `all_kernel_diagnoses.`
</div>

In [24]:
def all_kernel_diagnoses(conflicts):
    kernel_diagnoses = []
    for c in conflicts:
        temp = update_kernel_diagnoses(kernel_diagnoses,c)
        kernel_diagnoses = []
        for i in temp:
            kernel_diagnoses.append(i)
    return kernel_diagnoses
    

In [25]:
test_all_kernel_diagnoses(all_kernel_diagnoses)

Test passed!!